In [3]:
#load up modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import pickle
import os

import category_encoders as ce

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
#Get dir
path="D:/Kuliah/Semester 5/PSD/psd-challenge/"
os.listdir(path)

['Kerja lah su',
 'Metadata.xlsx',
 'Sample Submission - Copy.csv',
 'Sample Submission.csv',
 'Test.xlsx',
 'Train.xlsx',
 '~$Metadata.xlsx',
 '~$Test.xlsx',
 '~$Train.xlsx']

In [5]:
def savefile(result, savename):
    try:
        pd.DataFrame(zip(range(1,len(result)+1),result)).to_csv(path+"Kerja lah su/"+savename+".csv")
    except:
        print("It only saves the result retard!")

In [42]:
#Get dataset                                #Gw baru nyadar train di split lagi?
test_data=pd.read_excel(path+"Test.xlsx")
train=pd.read_excel(path+"Train.xlsx")

'''
#remove missing value
for i in [9,99]:
    test_data=test_data[test_data!=i].dropna(axis=0)
    train=train[train!=i].dropna(axis=0)
'''

'\n#remove missing value\nfor i in [9,99]:\n    test_data=test_data[test_data!=i].dropna(axis=0)\n    train=train[train!=i].dropna(axis=0)\n'

In [106]:
#Get Metadatadata and each attribut encoder
Metadata=pd.read_excel(path+"Metadata.xlsx")

# colnames w/ encoders
temp=[]
for i in Metadata.iloc[23:,0].unique():
    temp.append(i) if str(i).startswith("PSTV") else 0
temp

valueencoder={}
try:
    for i in range(len(temp)):
        temp1=Metadata.iloc[Metadata.iloc[23:,:][Metadata["Variable Information"]==temp[i]].index[0]:Metadata.iloc[23:,:][Metadata["Variable Information"]==temp[i+1]].index[0],:].values
        valueencoder[str(temp[i])] = (dict(zip(temp1[:,1], temp1[:,2])))
except:
    temp1=Metadata.iloc[Metadata.iloc[23:,:][Metadata["Variable Information"]=="PSTV17"].index[0]:,:].values
    valueencoder[str(temp[i])] = (dict(zip(temp1[:,1], temp1[:,2])))

C:\Users\Rhey\AppData\Local\Temp\ipykernel_7740\2399741515.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp1=Metadata.iloc[Metadata.iloc[23:,:][Metadata["Variable Information"]==temp[i]].index[0]:Metadata.iloc[23:,:][Metadata["Variable Information"]==temp[i+1]].index[0],:].values
C:\Users\Rhey\AppData\Local\Temp\ipykernel_7740\2399741515.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp1=Metadata.iloc[Metadata.iloc[23:,:][Metadata["Variable Information"]=="PSTV17"].index[0]:,:].values


In [43]:
X=train.iloc[:,:-1]
y=train.iloc[:,-1]

In [62]:
train_data, validation_data, train_value, validation_value= train_test_split(X, y, train_size=0.8)
model= LinearRegression().fit(train_data, train_value)

print(f'Train   :   {r2_score(train_value, model.predict(train_data))}')
print(f'Test    :   {r2_score(validation_value, model.predict(validation_data))}')

Train   :   0.23513284417345437
Test    :   0.24253254491203358


Diencode lebih buruk nilainya.

In [123]:
model= LinearRegression().fit(X, y)
predicred=model.predict(test_data)
savefile(predicred, "1Linreg")

#score 0.109

In [163]:
#Sorted
    #decode
feature_name=X.columns

In [167]:
#decoder
encodedtrain=train.copy()
encodedtest=test_data.copy()
objectcolname=[]
for i in feature_name:
    try:
        for j in encodedtrain[i].unique():
            encodedtrain[i][encodedtrain[i]==j]=valueencoder[i][j]
            encodedtest[i][encodedtest[i]==j]=encodedtest[i][j]
        objectcolname.append(i)
    except:
        pass

C:\Users\Rhey\AppData\Local\Temp\ipykernel_7740\1259356089.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encodedtrain[i][encodedtrain[i]==j]=valueencoder[i][j]


In [168]:
encodedtrain=encodedtrain.sort_values(by="UHH", ascending=True)

In [169]:
encoder=ce.OrdinalEncoder(cols=objectcolname)
encoder.fit(encodedtrain)
X=encoder.transform(encodedtrain)
encodedtrain=encoder.transform(encodedtrain)

In [171]:
X=encodedtrain
y=X.pop("UHH")

In [209]:
train_data, validation_data, train_value, validation_value= train_test_split(X, y, train_size=0.8)
model= LinearRegression().fit(train_data, train_value)

print(f'Train   :   {r2_score(train_value, model.predict(train_data))}')
print(f'Test    :   {r2_score(validation_value, model.predict(validation_data))}')

Train   :   0.11839029469381346
Test    :   0.10169318837202457


Hilangkan Missing Value (Lebih buruk)

In [97]:
for i in [9,99]:
    test_data=test_data[test_data!=i].dropna(axis=0)
    train=train[train!=i].dropna(axis=0)

In [98]:
X=train.iloc[:,:-1]
y=train.iloc[:,-1]

In [129]:
train_data, validation_data, train_value, validation_value= train_test_split(X, y, train_size=0.8)
model= LinearRegression().fit(train_data, train_value)

print(f'Train   :   {r2_score(train_value, model.predict(train_data))}')
print(f'Test    :   {r2_score(validation_value, model.predict(validation_data))}')

Train   :   0.2290547892645508
Test    :   0.21097418728633577
